Bu notebook eğitilmiş en iyi ağaç modelini kolay kullanılabilir bir kullanıcı arayüzüne aktarmak için oluşturulmuştur.

# Setup

In [1]:
import gradio as gr
import sklearn
import numpy as np
import json

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# PHASE 1: Get the model

Bu kısımda modellerin mimarisi tanımlanıp -asıl eğitildikleri notebook'takinin aynısı- kaydedilen modeller notebook'a yüklenir

In [2]:
import pickle

# Modeli yükle
with open(r"MODELS\catboost_model.pkl", 'rb') as f:
    cat_model = pickle.load(f)

cat_model

# PHASE 2: Configure the IU

Üç tane ana kullanıcı arayüzü tanımladım:
1) Single Prediction: Girilen verilere ve seçilen ayarlara göre tek bir zaman noktası tahmini yapar
2) Forecast Plot: Girilen verilere ve seçilen ayarlara göre istenilen aralığı öngörür ve grafiğini çıkartır
3) Forecast Table: Girilen verilere ve seçilen ayarlara göre istenilen aralığı öngörür ve tablosunu çıkartır

Her arayüz için kullanıcın ne yapması gerektiğini iyice anlatmaya çalıştım, arayüzün olduğu bölümü çalıştırırsanız açıklamaları daha net görebilirsiniz.

In [3]:
# Kullanıcının cevabını istenilen değişkene dönüştürmek için sözlük
dayofweek = {
    "Pazartesi": 1,
    "Salı": 2,
    "Çarşamba": 3,
    "Perşembe": 4,
    "Cuma": 5,
    "Cumartesi": 6,
    "Pazar": 7
}

## Predict Single

### Prediction logic

In [4]:
def predict(hour_interval, day_of_week, value_before):
    sequence = np.array([hour_interval, dayofweek[day_of_week], value_before])
    sequence_df = pd.DataFrame(sequence.reshape(1, -1), columns=["HourInterval", "DayOfWeek", "NetAmountWOVat Before 1"])

    output = cat_model.predict(sequence_df)
    
    return round(output.item(), 2)

### UI

In [5]:
single_article = "**Nasıl Kullanılır**:\n"\
            "* Tahmin etmek istediğiniz saati girin, günü seçin ve bir saat öncesindeki değeri sağlayın.\n" \
            "* 'Submit' düğmesine tıklayın. Tahmin edilen değer bölümü istenen sonucu gösterecektir. "

In [6]:
prediction_interface = gr.Interface( 
    fn=predict,
    inputs=[
        gr.Number(label="Saat Aralığı", value=14),
        gr.Dropdown(choices= ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"], label="Haftanın Günü", value="Cuma"),
        gr.Number(label="Bir önceki değer", value=30000)
    ],
    outputs=gr.Textbox(label="Tahmin edilen değer"),
    title="Öngörü Uygulaması",
    description="Model Türü, Saat Aralığı, Haftanın Günü ve Önceki Değer'i girerek bir tahmin alın.",
    theme=gr.themes.Soft(),
    article=single_article,
    allow_flagging="never"
)

prediction_interface.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


## Forecast Plot

### Forecasting Logic

In [7]:
desired_column = "NetAmountWOVat"
features = ["HourInterval", "DayOfWeek", f'{desired_column} Before 1']

def forecast_plot(start_timestamp, end_timestamp, sequence_length=1):
    forecast_timestamps = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')
    forecast_df = pd.DataFrame(index=forecast_timestamps)
    
    forecast_df[f'Forecasted {desired_column}'] = np.nan
    forecast_df["HourInterval"] = forecast_df.index.hour
    forecast_df['DayOfWeek'] = forecast_df.index.dayofweek + 1

    for i in range(1, sequence_length+1):
        forecast_df[f'{desired_column} Before {i}'] = np.nan

    forecast_df.iloc[:sequence_length, forecast_df.columns.get_loc(f'{desired_column} Before 1')] = 0

    for i in range(0, len(forecast_df) + 100):
        input_array = forecast_df[features].iloc[i]
        input_df = pd.DataFrame([input_array])

        predicted_value = cat_model.predict(input_df)
    
        forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')] = predicted_value.item()
    
        if forecast_df.isnull().sum()[0] == 0:
            break
    
        for k in range(1, sequence_length+1):
            forecast_df.iloc[i + k, forecast_df.columns.get_loc(f'{desired_column} Before {k}')] = forecast_df.iloc[i+1-k, forecast_df.columns.get_loc(f'Forecasted {desired_column}')]
    
    return px.line(forecast_df, x=forecast_df.index, y=f'Forecasted {desired_column}')

### UI

In [8]:
plot_article = "Nasıl Kullanılır:\n * Başlangıç ve bitiş tarihini YY-AA-GG SS:DD:SS biçiminde girin ve geçmişe bakma aralığını sağlayın. \n"\
                "* 'Submit' düğmesine tıklayın. Tahminler bir grafik olarak gösterilecektir. "

In [9]:
plot_interface = gr.Interface(
    fn=forecast_plot,
    inputs=[
        gr.Textbox(label="Başlangıç Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-8 00:00:00"),  
        gr.Textbox(label="Bitiş Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-15 23:00:00")
      ],
    outputs="plot",
    title="Tahmin Uygulaması",
    description="Tahmin yapmak için başlangıç ve bitiş zamanını YYYY-AA-GG SS:DD:SS formatında girin.",
    article = plot_article,
    allow_flagging="never"
)

plot_interface.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


## Forecast Table

### Forecasting Logic

In [10]:
desired_column = "NetAmountWOVat"
features = ["HourInterval", "DayOfWeek", f'{desired_column} Before 1']

def forecast_table(start_timestamp, end_timestamp, sequence_length=1):
    forecast_timestamps = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')
    forecast_df = pd.DataFrame(index=forecast_timestamps)
    
    forecast_df[f'Forecasted {desired_column}'] = np.nan
    forecast_df["HourInterval"] = forecast_df.index.hour
    forecast_df['DayOfWeek'] = forecast_df.index.dayofweek + 1

    for i in range(1, sequence_length+1):
        forecast_df[f'{desired_column} Before {i}'] = np.nan

    forecast_df.iloc[:sequence_length, forecast_df.columns.get_loc(f'{desired_column} Before 1')] = 0

    for i in range(0, len(forecast_df) + 100):
        input_array = forecast_df[features].iloc[i]
        input_df = pd.DataFrame([input_array])

        predicted_value = cat_model.predict(input_df)
    
        forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')] = predicted_value.item()
    
        if forecast_df.isnull().sum()[0] == 0:
            break
    
        for k in range(1, sequence_length+1):
            forecast_df.iloc[i + k, forecast_df.columns.get_loc(f'{desired_column} Before {k}')] = forecast_df.iloc[i+1-k, forecast_df.columns.get_loc(f'Forecasted {desired_column}')]

    formatted_timestamps = forecast_df.index.strftime("%Y-%m-%d %H:%M:%S").tolist()
    predicted_values = forecast_df[f'Forecasted {desired_column}'].tolist()

    data = list(zip(formatted_timestamps, predicted_values))

    return gr.Dataframe(value=data, headers=["Time", f"Predicted {desired_column}"])

### UI

In [11]:
table_article = "Nasıl Kullanılır:\n * Başlangıç ve bitiş tarihini YY-AA-GG SS:DD:SS biçiminde girin ve geçmişe bakma aralığını sağlayın. \n"\
                "* 'Submit' düğmesine tıklayın. Tahminler tablo halinde gösterilecektir. "

In [12]:
interface = gr.Interface(
    fn=forecast_table,
    inputs=[
        gr.Textbox(label="Başlangıç Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-8 00:00:00"),  
        gr.Textbox(label="Bitiş Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-10 23:00:00")
      ],
    outputs=gr.Dataframe(label="Tahmin Tablosu"),
    title="Tahmin Uygulaması",
    description="Tahmin yapmak için başlangıç ve bitiş zamanını YYYY-AA-GG SS:DD:SS formatında girin.",
    article=table_article,
    allow_flagging="never"
)

interface.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\kerem.cantimur\AppData\Local\anaconda3\envs\WorkEnv\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\kerem.cantimur\AppData\Local\anaconda3\envs\WorkEnv\lib\asyncio\proactor_events.py", line 162, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] Varolan bir bağlantı uzaktaki bir ana bilgisayar tarafından zorla kapatıldı
